In [8]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples

import pandas as pd
import numpy as np
import polars as pl
import datetime
import seaborn as sns
import lightgbm as lgb
import random
import matplotlib.pyplot as plt
import logging
import time
import re

In [9]:
params = {
    "daysAfterPrediction": None,
    "idxAfterPrediction": 5,
    'timesteps': 60,
    'target_option': 'mean',
    
    "LoadupSamples_tree_scaling_standard": True,
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
}

In [10]:
timegroup = "group_regOHLCV_over5years"
treegroup = "group_debug"

eval_date = datetime.date(year=2025, month=6, day=13)
evaldates = [eval_date - datetime.timedelta(days=i) for i in range(1, 6)]
start_train_date = datetime.date(year=2023, month=1, day=1)

ls = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=timegroup,
    params=params,
)
lstree = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=treegroup,
    timegroup=None,
    params=params,
)
lstime = LoadupSamples(
    train_start_date=start_train_date,
    test_dates=evaldates,
    treegroup=None,
    timegroup=timegroup,
    params=params,
)

In [11]:
ls.load_samples(main_path = "../src/featureAlchemy/bin/")
lstree.load_samples(main_path = "../src/featureAlchemy/bin/")
lstime.load_samples(main_path = "../src/featureAlchemy/bin/")

Test date 2025-06-08 not found in the database. Omitting.
Test date 2025-06-08 not found in the database. Omitting.
Test date 2025-06-08 not found in the database. Omitting.


In [12]:
train_Xtree = ls.train_Xtree
train_ytree = ls.train_ytree
train_Xtime = ls.train_Xtime
train_ytime = ls.train_ytime

test_Xtree = ls.test_Xtree
test_ytree = ls.test_ytree
test_Xtime = ls.test_Xtime
test_ytime = ls.test_ytime

treenames = ls.featureTreeNames
timenames = ls.featureTimeNames

meta_pl_train = ls.meta_pl_train
meta_pl_test = ls.meta_pl_test

print(np.mean(train_ytree)) if train_ytree.any() else None
print(np.mean(train_ytime)) if train_ytime.any() else None
print(timenames)
print(train_Xtime.shape)

1.0040375684891127
0.5167194715034465
[np.str_('FeatureLSTM_Price'), np.str_('FeatureLSTM_Volume')]
(11052, 60, 2)


In [13]:
split_Date = datetime.date(year=2025, month=1, day=1)
ls.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)
lstree.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)
lstime.split_dataset(
    start_date=start_train_date,
    last_train_date=split_Date,
    last_test_date=eval_date
)

In [14]:
mask_train = np.ones((ls.train_Xtree.shape[0],), dtype=bool)
mask_test = np.ones((ls.test_Xtree.shape[0],), dtype=bool)
rat = 0.1
mask_train = mask_train & (np.random.rand(ls.train_Xtree.shape[0]) <= rat)
mask_test  = mask_test  & (np.random.rand(ls.test_Xtree.shape[0]) <= rat)

ls.apply_masks(mask_train, mask_test)
assert ls.train_Xtree.shape[0] == np.sum(mask_train)
assert ls.test_Xtree.shape[0] == np.sum(mask_test)